In [245]:
import pandas as pd
import time
import math
import datetime
import glob
import matplotlib.pyplot as plt
import plotly.express as px

In [246]:
#def sparkline(data, figsize=(4, 0.25), **kwargs):
#    data = list(data)
#
#    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwargs)
#    ax.plot(data)
#    ax.fill_between(range(len(data)), data, len(data)*[min(data)], alpha=0.1)
#    ax.set_axis_off()
#    img = BytesIO()
#    plt.savefig(img)
#    plt.close()
#    return '<img src="data:image/png;base64, {}" />'.format(base64.b64encode(img.getvalue()).decode())

In [247]:
def getStockInfo(stock):
    ticker = stock
    period1 = int(time.mktime(datetime.datetime(2022,12,1,23,59).timetuple()))
    period2 = int(time.mktime(datetime.datetime(2023,1,15,23,59).timetuple()))
    interval = '1d'
    url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
    df=pd.read_csv(url)
    df = df.assign(ticker = ticker)
    os.makedirs('companies', exist_ok=True)  
    df.to_csv(f'companies/{ticker}.csv')

In [248]:
stocks = ["AAPL","DIS","TSLA"]
for i in stocks:
    getStockInfo(i)

In [249]:
path = '/Users/joshuaheine/Desktop/stockCalculator/companies'
csv_files = glob.glob(path + "/*.csv")

df_list = (pd.read_csv(file) for file in csv_files)

big_df = pd.concat(df_list, ignore_index=True)

In [250]:
#cleaning data 
big_df = big_df.loc[:, ~big_df.columns.str.contains('^Unnamed')]
big_df.columns.str.lower()

df2 = big_df.rename(columns = {'Date' :'stock_date', 'Open':'open_price', 'High':'high_price', 'Low':'low_price', 'Close':'close_price', 'Adj Close':'adj_close',
       'Volume':'volume', 'ticker':'ticker'}, inplace = False)
df2.head()

df2.isnull()

,stock_date,open_price,high_price,low_price,close_price,adj_close,volume,ticker
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
85,False,False,False,False,False,False,False,False
86,False,False,False,False,False,False,False,False
87,False,False,False,False,False,False,False,False
88,False,False,False,False,False,False,False,False


In [251]:
df2.isnull().sum()

stock_date     0
open_price     0
high_price     0
low_price      0
close_price    0
adj_close      0
volume         0
ticker         0
dtype: int64

In [252]:
df2.close_price = df2.close_price.fillna(0)
print(df2.close_price.isnull().sum())

0


In [253]:
df2 = df2.dropna() 
df2.isnull().sum()

stock_date     0
open_price     0
high_price     0
low_price      0
close_price    0
adj_close      0
volume         0
ticker         0
dtype: int64

In [254]:
print(df2)

    stock_date  open_price  high_price   low_price  close_price   adj_close  \
0   2022-12-01   97.870003   98.839996   96.580002    98.589996   98.589996   
1   2022-12-02   97.459999   99.690002   97.010002    99.430000   99.430000   
2   2022-12-05   98.239998   99.160004   95.669998    95.930000   95.930000   
3   2022-12-06   95.730003   96.000000   91.980003    92.290001   92.290001   
4   2022-12-07   92.660004   92.790001   91.260002    92.150002   92.150002   
..         ...         ...         ...         ...          ...         ...   
85  2023-01-09  130.470001  133.410004  129.889999   130.149994  130.149994   
86  2023-01-10  130.259995  131.259995  128.119995   130.729996  130.729996   
87  2023-01-11  131.250000  133.509995  130.460007   133.490005  133.490005   
88  2023-01-12  133.880005  134.259995  131.440002   133.410004  133.410004   
89  2023-01-13  132.029999  134.919998  131.660004   134.759995  134.759995   

      volume ticker  
0   13855700    DIS  
1   126

In [255]:
result = df2.dtypes

print("Output:")
print(result)

Output:
stock_date      object
open_price     float64
high_price     float64
low_price      float64
close_price    float64
adj_close      float64
volume           int64
ticker          object
dtype: object


In [256]:
df_apple = df2[df2['ticker'] == 'AAPL']
df_apple.describe()

,open_price,high_price,low_price,close_price,adj_close,volume
count,30.000000,30.000000,30.000000,30.000000,30.000000,3.000000e+01
mean,136.055001,137.615999,133.743001,135.467333,135.467333,7.929405e+07
std,7.457452,7.345171,7.176530,7.161378,7.161378,1.932464e+07
min,126.010002,127.769997,124.169998,125.019997,125.019997,5.775800e+07
25%,130.327500,131.297497,128.269996,130.059998,130.059998,6.912058e+07
50%,133.430001,134.739998,131.380004,133.450004,133.450004,7.590035e+07
75%,142.355000,144.255001,141.019997,142.527496,142.527496,8.465160e+07
max,149.500000,150.919998,146.610001,148.309998,148.309998,1.601569e+08


In [257]:
fig = px.line(df_apple, x = df_apple['stock_date'], y = df_apple['close_price'], title='''Apple's Share average within 1 month''')
fig.show()

In [258]:
df_apple['close_price'].mean()

135.46733286666665

In [259]:
df_dis = df2[df2['ticker'] == 'DIS']
df_dis.describe()

,open_price,high_price,low_price,close_price,adj_close,volume
count,30.000000,30.000000,30.000000,30.000000,30.000000,3.000000e+01
mean,91.978001,93.158001,90.635334,92.002000,92.002000,1.373906e+07
std,4.411751,4.359731,4.274558,4.491940,4.491940,4.530572e+06
min,85.250000,86.690002,84.070000,84.169998,84.169998,6.914600e+06
25%,87.809999,88.544998,86.090000,87.387501,87.387501,1.104650e+07
50%,92.485000,93.275002,91.290001,92.220001,92.220001,1.231000e+07
75%,95.432502,95.924999,93.539999,94.752497,94.752497,1.498712e+07
max,99.379997,100.629997,98.010002,99.809998,99.809998,2.638390e+07


In [260]:
fig2 = px.line(df_dis, x = df_dis['stock_date'], y = df_dis['close_price'], title='''Disney's Share average within 1 month''')
fig2.show()

In [261]:
df_dis['close_price'].mean()

92.0019997

In [262]:
df_tsla = df2[df2['ticker'] == 'TSLA']
df_tsla.describe()

,open_price,high_price,low_price,close_price,adj_close,volume
count,30.000000,30.000000,30.000000,30.000000,30.000000,3.000000e+01
mean,143.872000,147.195666,138.655334,142.177333,142.177333,1.541902e+08
std,29.206194,28.759536,28.657089,28.111084,28.111084,4.696274e+07
min,103.000000,111.750000,101.809998,108.099998,108.099998,7.364590e+07
25%,119.207498,122.950001,115.949998,120.282498,120.282498,1.129295e+08
50%,137.669998,138.945000,129.075000,131.460003,131.460003,1.587748e+08
75%,173.429996,175.162499,164.867504,166.102505,166.102505,1.829679e+08
max,197.080002,198.919998,191.800003,194.860001,194.860001,2.314028e+08


In [263]:
fig3 = px.line(df_tsla, x = df_tsla['stock_date'], y = df_tsla['close_price'], title='''Tesla's Share average within 1 month''')
fig3.show()

In [264]:
df_tsla['close_price'].mean()

142.17733300000003

In [265]:
#df3 = pd.DataFrame(df2,columns=['close_price','ticker','stock_date'])
#pvt = df3.pivot(index='ticker', columns='stock_date', values='close_price')
#pvt['month_trend'] = pvt.apply(sparkline,axis=1)